In [1]:
import numpy as np
import pandas as pd 
import re 
import gensim 
import json
import timeit
import matplotlib.pyplot as plt

In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

## Read patents

In [15]:
ids = pd.read_csv(r"../data/gp-q2-clean.csv",index_col='code',skipinitialspace=True)
print(ids.isnull().sum())
print('length is',len(ids))

title        0
pub_date     0
citations    0
abstract     0
class        0
year         0
dtype: int64
length is 9476


## Read articles

In [17]:
arts = pd.read_csv(r"../data/scopus-q2-clean.csv",index_col='code',skipinitialspace=True)
arts

,title,year,link,abstract,keyword
code,,,,,
85062908267,Terrorist network analysis and identification ...,2018,https://www.scopus.com/inward/record.uri?eid=2...,The prediction of terrorist network and identi...,Internet of things; Learning algorithms; Smart...
85092037876,Who makes revolution in the age of speculative...,2018,https://www.scopus.com/inward/record.uri?eid=2...,Contemporary theories of social emancipation c...,NaN
85061535070,One-shot learning approach for unknown malware...,2018,https://www.scopus.com/inward/record.uri?eid=2...,Early detection of new kinds of malware always...,Character recognition; Classification (of info...
85055699089,Regulating Artificial Intelligence Proposal fo...,2018,https://www.scopus.com/inward/record.uri?eid=2...,Given the ubiquity of artificial intelligence ...,Autonomous vehicles; International trade; Pers...
85061405847,Defending Against Adversarial Samples Without ...,2018,https://www.scopus.com/inward/record.uri?eid=2...,It has been recently shown that deep neural ne...,Computer crime; Data mining; Deep neural netwo...
...,...,...,...,...,...
85012264765,Mathematical fortune-telling,2001,https://www.scopus.com/inward/record.uri?eid=2...,This is an informal introduction to the physic...,NaN
77956055808,Selection of behavior in social situations app...,2001,https://www.scopus.com/inward/record.uri?eid=2...,The law of effect is a very simple law which r...,Computer science; Computers; Adaptive behavior...
35786351,Compactness measurement using Fuzzy Multicrite...,2001,https://www.scopus.com/inward/record.uri?eid=2...,This paper presents a new method for compactne...,Algorithms; Decision making; Decision support ...


# Topic Modeling

In [18]:
x_train = [abstract for abstract in ids.abstract]
x_test = [abstract for abstract in arts.abstract]

print(len(x_train),len(x_test))
#x_train = x_train.extend(x_test)






9476 7981


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

it_stopwords = stopwords.words('english')

tf_vectorizer = CountVectorizer(stop_words=it_stopwords, max_df=0.5, min_df=5,max_features = 1000, ngram_range=(1,2))
tf = tf_vectorizer.fit_transform(x_train)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 8

lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tf)


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=1)

In [21]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [22]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0: system, information, module, control, unit, invention, data, terminal, present, communication, vehicle, present invention, mobile, monitoring, method, image, management, signal, processing, server
Topic 1: user, device, authentication, information, service, client, server, access, request, mobile, devices, application, system, based, method, may, computing, electronic, users, response
Topic 2: one, least, first, least one, based, plurality, event, system, set, associated, second, method, includes, may, security, events, score, entity, data, determining
Topic 3: data, content, system, may, message, search, processing, query, key, based, one, method, storage, document, time, includes, set, using, first, encrypted
Topic 4: information, system, security, network, object, web, image, risk, domain, video, computer, based, systems, internet, objects, threat, management, database, method, website
Topic 5: computer, file, may, computing, application, include, disclosed, systems, method

In [23]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0: system, information, module, control, unit, invention, data, terminal, present, communication, vehicle, present invention, mobile, monitoring, method, image, management, signal, processing, server
Topic 1: user, device, authentication, information, service, client, server, access, request, mobile, devices, application, system, based, method, may, computing, electronic, users, response
Topic 2: one, least, first, least one, based, plurality, event, system, set, associated, second, method, includes, may, security, events, score, entity, data, determining
Topic 3: data, content, system, may, message, search, processing, query, key, based, one, method, storage, document, time, includes, set, using, first, encrypted
Topic 4: information, system, security, network, object, web, image, risk, domain, video, computer, based, systems, internet, objects, threat, management, database, method, website
Topic 5: computer, file, may, computing, application, include, disclosed, systems, method

In [24]:
doc_topics = lda.transform(tf)
doc_topics

array([[0.14409507, 0.55646959, 0.0487804 , ..., 0.00140566, 0.00141114,
        0.10314256],
       [0.00131722, 0.09726356, 0.07766972, ..., 0.00131836, 0.14949885,
        0.47326471],
       [0.00278073, 0.13884967, 0.00278316, ..., 0.00278573, 0.00278094,
        0.6064935 ],
       ...,
       [0.46225237, 0.00173752, 0.37231395, ..., 0.12003534, 0.00173891,
        0.0384441 ],
       [0.00255488, 0.00255604, 0.84535513, ..., 0.00255329, 0.13931576,
        0.00255299],
       [0.00338548, 0.18879345, 0.33897108, ..., 0.00338034, 0.45532425,
        0.003383  ]])

In [31]:
doc_topics = lda.transform(tf)
ids['topic'] = np.argmax(doc_topics,axis=1)
ids['topic'].value_counts()

2    1586
6    1475
0    1435
1    1125
5    1046
3    1026
4     974
7     809
Name: topic, dtype: int64

In [30]:
for a,b in zip(ids['topic'].value_counts(),arts['topic'].value_counts()): print(a,b)

1586 4628
1475 2101
1435 472
1125 298
1046 217
1026 131
974 97
809 37


In [35]:
for i in ids['topic'].value_counts().keys(): print(i,ids['topic'].value_counts()[i],arts['topic'].value_counts()[i])

2 1586 217
6 1475 4628
0 1435 298
1 1125 131
5 1046 37
3 1026 472
4 974 2101
7 809 97


In [27]:
tf2 = tf_vectorizer.transform(x_test)
arts_topics = lda.transform(tf2)


In [28]:
arts['topic'] = np.argmax(arts_topics,axis=1)
arts['topic'].value_counts()

6    4628
4    2101
3     472
0     298
2     217
1     131
7      97
5      37
Name: topic, dtype: int64

In [23]:
ids.to_csv('../data/gp-q2-topics.csv')
arts.to_csv('../data/scopus-q2-topics.csv')

# DOCS
here we use directy patet + papers

In [6]:
docs = pd.read_csv('../data/docs-clean.csv', index_col='code')
docs.head()

,title,year,abstract,doctype,class
code,,,,,
85062908267,Terrorist network analysis and identification ...,2018,the prediction of terrorist network and identi...,paper,NAN
85092037876,Who makes revolution in the age of speculative...,2018,contemporary theories of social emancipation c...,paper,NAN
85061535070,One-shot learning approach for unknown malware...,2018,early detection of new kinds of malware always...,paper,NAN
85055699089,Regulating Artificial Intelligence Proposal fo...,2018,given the ubiquity of artificial intelligence ...,paper,NAN
85061405847,Defending Against Adversarial Samples Without ...,2018,it has been recently shown that deep neural ne...,paper,NAN


In [8]:
ids = docs
ids = ids[ids.abstract.notna()]
x_train = [abstract for abstract in ids.abstract]

### SCikit Learn

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

it_stopwords = stopwords.words('english')

tf_vectorizer = CountVectorizer(stop_words=it_stopwords, max_df=0.5, min_df=5,max_features = 1000, ngram_range=(1,2))
tf = tf_vectorizer.fit_transform(x_train)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 8

lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tf)


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=1)

In [12]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0: security, cyber, systems, intelligence, malicious, software, code, attack, attacks, system, domain, artificial, threat, threats, new, artificial intelligence, virtual, paper, also, internet
Topic 1: news, network, detection, fake, based, fake news, networks, information, using, learning, paper, real, proposed, approach, model, traffic, research, system, algorithm, results
Topic 2: malware, model, based, network, classification, neural, detection, using, learning, deep, analysis, method, training, neural network, feature, iot, features, deep learning, data, image
Topic 3: data, device, one, network, first, may, least, based, system, second, associated, user, least one, plurality, computing, includes, devices, content, traffic, communication
Topic 4: user, information, system, data, service, module, method, authentication, server, invention, control, access, device, management, based, terminal, unit, present, vehicle, message
Topic 5: malware, detection, learning, machine, machi

In [13]:
doc_topics = lda.transform(tf)
ids['topic'] = np.argmax(doc_topics,axis=1)
ids.topic.value_counts()

3    3590
5    2947
4    2450
7    2258
1    1976
6    1749
0    1468
2    1019
Name: topic, dtype: int64

In [14]:
pd.crosstab(ids['doctype'],ids['topic'])

topic,0,1,2,3,4,5,6,7
doctype,,,,,,,,
paper,1143,1906,591,4,72,2800,1454,11
patent,325,70,428,3586,2378,147,295,2247


### GEnsim

In [7]:
import os
import sys
import gensim

from gensim import corpora
from gensim import models
from gensim.models.coherencemodel import CoherenceModel

import spacy
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

from spacy.lang.en.stop_words import STOP_WORDS

In [8]:
docs = pd.read_csv('../data/docs-clean.csv', index_col='code')
docs.head()

,title,year,abstract,doctype,class
code,,,,,
85062908267,Terrorist network analysis and identification ...,2018,the prediction of terrorist network and identi...,paper,NAN
85092037876,Who makes revolution in the age of speculative...,2018,contemporary theories of social emancipation c...,paper,NAN
85061535070,One-shot learning approach for unknown malware...,2018,early detection of new kinds of malware always...,paper,NAN
85055699089,Regulating Artificial Intelligence Proposal fo...,2018,given the ubiquity of artificial intelligence ...,paper,NAN
85061405847,Defending Against Adversarial Samples Without ...,2018,it has been recently shown that deep neural ne...,paper,NAN


In [10]:
x_train = docs['abstract'].values

In [11]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS

texts = [[w for w in text.split(' ') if w not in stopwords] for text in x_train] #tokenization


In [13]:

dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

tfidf = models.TfidfModel(corpus) 

corpus_tfidf = tfidf[corpus]

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [47]:
topics = [scores[np.argmax([score[1] for score in scores])][0] for scores in lda_model.get_document_topics(corpus) ]

In [49]:
for i in range(8): 
    print('TOPIC ',i,': ', lda_model.get_topic_terms(i, topn=10))
   

TOPIC  0 :  [(656, 0.07298501), (3511, 0.06786042), (1641, 0.026658064), (923, 0.023507943), (805, 0.019785155), (767, 0.019587766), (648, 0.018938249), (2408, 0.01839882), (1645, 0.017397333), (546, 0.015544288)]
TOPIC  1 :  [(1901, 0.108384036), (1865, 0.097540714), (21782, 0.091617346), (7699, 0.067549355), (400, 0.031842597), (41626, 0.020621914), (6791, 0.020197736), (41572, 0.016224992), (4109, 0.015916632), (4978, 0.0121265035)]
TOPIC  2 :  [(322, 0.035775743), (886, 0.025024831), (460, 0.023265406), (4402, 0.019195244), (1166, 0.016796492), (1761, 0.015895726), (842, 0.01578771), (3971, 0.012511091), (4806, 0.011436595), (1080, 0.011002939)]
TOPIC  3 :  [(282, 0.04785681), (38, 0.029460937), (169, 0.026424186), (0, 0.024500204), (24, 0.017419834), (35, 0.0160064), (439, 0.013133688), (1562, 0.009949418), (7, 0.008941776), (964, 0.0083625)]
TOPIC  4 :  [(6276, 0.07324232), (4853, 0.03355926), (1187, 0.02791631), (4015, 0.023547702), (879, 0.01818989), (42824, 0.015513118), (3750

In [50]:
lda_model.print_topics(num_topics=8, num_words=10)

[(0,
  '0.073*"content" + 0.068*"receiving" + 0.027*"flow" + 0.024*"domain" + 0.020*"graph" + 0.020*"search" + 0.019*"web" + 0.018*"obtaining" + 0.017*"list" + 0.016*"document"'),
 (1,
  '0.108*")" + 0.098*"(" + 0.092*"invention" + 0.068*"plurality" + 0.032*"media" + 0.021*"computer-readable" + 0.020*"apparatus" + 0.016*"computer-implemented" + 0.016*"query" + 0.012*"said"'),
 (2,
  '0.036*"module" + 0.025*"control" + 0.023*"image" + 0.019*"vehicle" + 0.017*"object" + 0.016*"video" + 0.016*"signal" + 0.013*"wireless" + 0.011*"sensor" + 0.011*"monitoring"'),
 (3,
  '0.048*"data" + 0.029*"system" + 0.026*"information" + 0.025*"" + 0.017*"method" + 0.016*"security" + 0.013*"based" + 0.010*"second" + 0.009*"computing" + 0.008*"includes"'),
 (4,
  '0.073*"electronic" + 0.034*"storing" + 0.028*"id" + 0.024*"included" + 0.018*"assets" + 0.016*"der" + 0.014*"entry" + 0.014*"responsive" + 0.013*"des" + 0.012*"un"'),
 (5,
  '0.104*"device" + 0.082*"user" + 0.076*"network" + 0.029*"server" + 0.02

In [51]:
pd.Series(topics).value_counts()

3    11151
6     5445
5      277
1      274
2      211
0       55
4       40
7        4
dtype: int64

In [ ]:
patents = pd.read_csv